Start!

In [1]:
!pip install timesfm[torch]
!pip install 'apache_beam[interactive,gcp,test] == 2.67.0'
!pip install google-generativeai


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: "'apache_beam[interactive,gcp,test]": Expected package name at the start of dependency specifier
    'apache_beam[interactive,gcp,test]
    ^


  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
  Using cached google_api_core-2.28.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached google_api_python_client-2.187.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached google_auth-2.43.0-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached googleapis_common_protos-1.72.0-py3-none-any.whl.metadata (9.4 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached httplib2-0.31.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached google_auth_httplib2-0.2.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached uritemplate-4.2.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached grpcio_status-1.76.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.m


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import sys, apache_beam as beam
print("Python:", sys.version)
print("Beam:", beam.__version__)

Failed to import GCSFileSystem; loading of this filesystem will be skipped. Error details: cannot import name 'storage' from 'google.cloud' (unknown location)


Python: 3.11.14 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 18:30:03) [MSC v.1929 64 bit (AMD64)]
Beam: 2.67.0


In [12]:
import logging, bisect
import apache_beam as beam
from apache_beam.coders import BooleanCoder, PickleCoder, TimestampCoder
from apache_beam.transforms.timeutil import TimeDomain
from apache_beam.transforms.userstate import ReadModifyWriteStateSpec, TimerSpec, on_timer
from apache_beam.utils.timestamp import MAX_TIMESTAMP, Timestamp

# Try to import OrderedListStateSpec; fall back if unavailable.
try:
    from apache_beam.transforms.userstate import OrderedListStateSpec  
    _HAS_ORDERED_LIST = True
except Exception:
    _HAS_ORDERED_LIST = False

_LOGGER = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
_LOGGER.setLevel(logging.INFO)

class OrderedSlidingWindowFn(beam.DoFn):
  # Use OrderedListState when available; otherwise store a sorted Python list via ReadModifyWriteState
  if _HAS_ORDERED_LIST:
    ORDERED_BUFFER_STATE = OrderedListStateSpec('ordered_buffer', PickleCoder())
  else:
    ORDERED_BUFFER_STATE = ReadModifyWriteStateSpec('ordered_buffer', PickleCoder())

  WINDOW_TIMER = TimerSpec('window_timer', TimeDomain.WATERMARK)
  TIMER_STATE = ReadModifyWriteStateSpec('timer_state', BooleanCoder())
  EARLIEST_TS_STATE = ReadModifyWriteStateSpec('earliest_ts', TimestampCoder())

  def __init__(self, window_size, slide_interval):
    # window_size, slide_interval are in seconds (ints/floats)
    self.window_size = window_size
    self.slide_interval = slide_interval

  def start_bundle(self):
    _LOGGER.debug("start bundle")

  def finish_bundle(self):
    _LOGGER.debug("finish bundle")

  def _insert_sorted(self, buf, ts, val):
    
    # sort by microseconds to avoid relying on Timestamp rich-compare on all platforms
    keys = [int(t.micros) for t, _ in buf]
    pos = bisect.bisect_right(keys, int(ts.micros))
    buf.insert(pos, (ts, val))
    return buf

  def process(
      self,
      element,
      timestamp=beam.DoFn.TimestampParam,
      ordered_buffer=beam.DoFn.StateParam(ORDERED_BUFFER_STATE),
      window_timer=beam.DoFn.TimerParam(WINDOW_TIMER),
      timer_state=beam.DoFn.StateParam(TIMER_STATE),
      earliest_ts_state=beam.DoFn.StateParam(EARLIEST_TS_STATE)):

    _, value = element

    if _HAS_ORDERED_LIST:
      ordered_buffer.add((timestamp, value))
    else:
      buf = ordered_buffer.read() or []
      buf = self._insert_sorted(buf, timestamp, value)
      ordered_buffer.write(buf)

    _LOGGER.debug("receive %s at %s", element, timestamp)
    timer_started = timer_state.read()

    earliest = earliest_ts_state.read()
    if not earliest or earliest > timestamp:
      earliest_ts_state.write(timestamp)

    if not timer_started:
      earliest_ts_state.write(timestamp)
      first_slide_start = int(timestamp.micros / 1e6 // self.slide_interval) * self.slide_interval
      first_slide_start_ts = Timestamp.of(first_slide_start)
      first_window_end_ts = first_slide_start_ts + self.window_size
      _LOGGER.debug("set timer to %s", first_window_end_ts)
      window_timer.set(first_window_end_ts)
      timer_state.write(True)

    return []

  @on_timer(WINDOW_TIMER)
  def on_timer(
      self,
      key=beam.DoFn.KeyParam,
      fire_ts=beam.DoFn.TimestampParam,
      ordered_buffer=beam.DoFn.StateParam(ORDERED_BUFFER_STATE),
      window_timer=beam.DoFn.TimerParam(WINDOW_TIMER),
      timer_state=beam.DoFn.StateParam(TIMER_STATE),
      earliest_ts_state=beam.DoFn.StateParam(EARLIEST_TS_STATE)):

    _LOGGER.debug("timer fire at %s", fire_ts)
    window_end_ts = fire_ts
    window_start_ts = window_end_ts - self.window_size

    if _HAS_ORDERED_LIST:
      window_values = list(ordered_buffer.read_range(window_start_ts, window_end_ts))
    else:
      buf = ordered_buffer.read() or []
      # take items in [window_start_ts, window_end_ts)
      window_values = [(ts, val) for (ts, val) in buf if window_start_ts <= ts < window_end_ts]

    _LOGGER.debug("window start: %s, window end: %s", window_start_ts, window_end_ts)
    _LOGGER.debug("windowed data in buffer %s", str(window_values))
    if window_values:
      yield (key, (window_start_ts, window_end_ts, window_values))

    next_window_end_ts = fire_ts + self.slide_interval
    next_window_start_ts = window_start_ts + self.slide_interval

    if _HAS_ORDERED_LIST:
      earliest_ts = earliest_ts_state.read()
      ordered_buffer.clear_range(earliest_ts, next_window_start_ts)
      remaining_data = list(ordered_buffer.read_range(next_window_start_ts, MAX_TIMESTAMP))
      if not remaining_data:
        timer_state.clear()
        earliest_ts_state.write(next_window_start_ts)
        return
      _LOGGER.debug("set timer to %s", next_window_end_ts)
      window_timer.set(next_window_end_ts)
    else:
      # prune older-than next_window_start_ts and write back
      buf = ordered_buffer.read() or []
      buf = [(ts, val) for (ts, val) in buf if ts >= next_window_start_ts]
      ordered_buffer.write(buf)
      if not buf:
        timer_state.clear()
        earliest_ts_state.write(next_window_start_ts)
        return
      _LOGGER.debug("set timer to %s", next_window_end_ts)
      window_timer.set(next_window_end_ts)

class FillGapsFn(beam.DoFn):
  def __init__(self, expected_interval: float):
    self.expected_interval = expected_interval  # seconds

  def process(self, element):
    key, (window_start_ts, window_end_ts, window_elements) = element
    received_data = {round(float(ts.micros / 1e6), 5): val for ts, val in window_elements}
    start_sec = float(window_start_ts.micros / 1e6)
    end_sec = float(window_end_ts.micros / 1e6)

    filled_values = []
    current_ts_sec = start_sec
    while current_ts_sec < end_sec:
      lookup_ts = round(current_ts_sec, 5)
      filled_values.append(float(received_data[lookup_ts]) if lookup_ts in received_data else 'NaN')
      current_ts_sec += self.expected_interval

    yield (key, (window_start_ts, window_end_ts, filled_values))


In [13]:
import apache_beam as beam
from apache_beam.ml.inference.base import ModelHandler
import timesfm
import logging
import numpy as np
import os
from google.cloud import storage
from apache_beam.io.gcp.gcsio import GcsIO
from apache_beam.utils.timestamp import Timestamp

class LatestModelCheckpointLoader(beam.PTransform):
    """A PTransform that finds the latest model checkpoint in a GCS path."""
    def __init__(self, gcs_bucket, gcs_prefix):
        self.gcs_bucket = gcs_bucket
        self.gcs_prefix = gcs_prefix

    def expand(self, pcoll):
        return pcoll | "FindLatestModel" >> beam.Map(self._find_latest_model_path)

    def _find_latest_model_path(self, _):
        try:
            storage_client = storage.Client()
            blobs = storage_client.list_blobs(self.gcs_bucket, prefix=self.gcs_prefix)
            # Filter for model files and find the most recent one
            model_blobs = [b for b in blobs if b.name.endswith(".pth")]
            latest_blob = max(model_blobs, key=lambda b: b.time_created, default=None)

            if latest_blob:
                path = f"gs://{self.gcs_bucket}/{latest_blob.name}"
                logging.info(f"Found latest finetuned model at: {path}")
                return path
        except Exception as e:
            logging.error(f"Error finding latest model in GCS: {e}")

        # Return a path to the base model if no finetuned one exists or an error occurs
        base_model = "google/timesfm-1.0-200m-pytorch"
        logging.info(f"No finetuned model found. Using base model: {base_model}")
        return base_model

class DynamicTimesFmModelHandler(ModelHandler[np.ndarray, np.ndarray, timesfm.TimesFm]):
    """
    A model handler that loads a TimesFM model from a dynamic path (GCS or Hugging Face).
    The model path is provided as a side input to RunInference.
    """
    def __init__(self, model_uri: str, hparams):
        self._hparams = hparams
        self._model = None
        self._model_uri = model_uri
        self._context_len = hparams.context_len
        self._horizon_len = hparams.horizon_len

    def load_model(self) -> timesfm.TimesFm:
        """Loads a model from the handler's current model_uri."""
        logging.info(f"Loading TimesFM model from path: {self._model_uri}...")

        checkpoint_config = {}
        if self._model_uri.startswith("gs://"):
            try:
                gcs = GcsIO()
                file_name = os.path.basename(self._model_uri)
                local_path = f"/tmp/{file_name}"
                with gcs.open(self._model_uri, 'rb') as f_in, open(local_path, 'wb') as f_out:
                    f_out.write(f_in.read())
                checkpoint_config['path'] = local_path
                logging.info(f"Downloaded model from GCS to {local_path}")
            except Exception as e:
                logging.error(f"Failed to download model from GCS: {e}. Check path and permissions.")
                raise e # Re-raise the exception to fail fast if the model can't be loaded.
        else:
            checkpoint_config['huggingface_repo_id'] = self._model_uri

        self._model = timesfm.TimesFm(
            hparams=self._hparams,
            checkpoint=timesfm.TimesFmCheckpoint(**checkpoint_config)
        )
        logging.info("TimesFM model loaded successfully.")
        return self._model

    def update_model_path(self, model_path: str):
        """
        This method is called by RunInference when a new model metadata is available
        from the side input. It updates the model URI that `load_model` will use.
        """
        if not model_path:
            logging.info("Received an empty model path update. No action taken.")
            return
        logging.info(f"Received model update. New model URI: {model_path}")
        self._model_uri = model_path
        self._model = self.load_model()
        logging.info("Model has been updated in the handler.")

    def run_inference(self, batch, model, inference_args=None):
        """
            Runs inference on a batch of data.

            Note: While this is a standard method for ModelHandler, we will call the
            model's `forecast` method directly in our DoFn for clarity.
            """
        # print("Running inference on batch:", batch)
        # logging.info(f"Running inference on batch:", batch)

        anomalies_found = []

        key, (window_start_ts, _, values_array) = batch[0]

        # A window must have enough data for both context and horizon.
        # if len(values_array) < self.context_len + self.horizon_len:
        #     return

        current_context = np.array(values_array[:self._context_len])
        actual_horizon_values = np.array(
            values_array[self._context_len:self._context_len + self._horizon_len])

        print("Current context shape:", current_context.shape)
        print("Actual horizon values shape:", actual_horizon_values.shape)
        point_forecast, experimental_quantile_forecast = model.forecast(
            [current_context],
            freq=[0],
        )

        current_predicted_horizon_values = point_forecast[
            0, :, 0] if point_forecast.ndim == 3 else point_forecast[0]

        current_q20_values = experimental_quantile_forecast[0, :, 2]
        current_q30_values = experimental_quantile_forecast[0, :, 3]
        current_q70_values = experimental_quantile_forecast[0, :, 7]
        current_q80_values = experimental_quantile_forecast[0, :, 8]

        for j in range(len(actual_horizon_values)):
            current_actual = actual_horizon_values[j]

            point_Q1 = np.nanmean([current_q20_values[j], current_q30_values[j]])
            point_Q3 = np.nanmean([current_q70_values[j], current_q80_values[j]])
            point_IQR = point_Q3 - point_Q1

            upper_thresh = point_Q3 + 1.5 * point_IQR
            lower_thresh = point_Q1 - 1.5 * point_IQR

            if current_actual > upper_thresh or current_actual < lower_thresh:
                score = (current_actual - upper_thresh
                            ) / point_IQR if current_actual > upper_thresh else (
                                lower_thresh - current_actual) / point_IQR

                anomaly_timestamp_seconds = (window_start_ts.micros / 1e6) + (
                    self._context_len + j)

                index_in_window = self._context_len + j

                anomalies_found.append({
                    'key': key,
                    'timestamp': Timestamp(anomaly_timestamp_seconds),
                    'index_in_window': index_in_window,
                    'actual_value': current_actual,
                    'predicted_value': current_predicted_horizon_values[j],
                    'is_anomaly': True,
                    'outlier_score': score,
                    'lower_bound': lower_thresh,
                    'upper_bound': upper_thresh,
                })
        payload = {
            "start_ts_micros": window_start_ts.micros,
            "predicted_values": current_predicted_horizon_values.tolist(),
            "q20_values": current_q20_values.tolist(),
            "q30_values": current_q30_values.tolist(),
            "q70_values": current_q70_values.tolist(),
            "q80_values": current_q80_values.tolist(),
            "anomalies": anomalies_found,  # Your original list is now inside the dictionary
            "actual_horizon_values": actual_horizon_values.tolist()
        }
        result_with_context = (batch[0], payload)

        return [result_with_context]
     

ModuleNotFoundError: No module named 'timesfm'